In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
from pprint import pprint

In [6]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")

sparql.setQuery("""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX dbp: <http://dbpedia.org/property/>

    SELECT ?res (MAX(?pop) as ?pop)
    WHERE {<http://dbpedia.org/resource/New_York_(state)> rdfs:comment ?res.
            <http://dbpedia.org/resource/New_York_(state)> dbp:pop ?pop.
            FILTER(LANG(?res)="en")
    }
"""
)

sparql.setReturnFormat(JSON)

result = sparql.query().convert()


In [7]:
pprint(result)

{'head': {'link': [], 'vars': ['res', 'pop']},
 'results': {'bindings': [{'pop': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer',
                                   'type': 'typed-literal',
                                   'value': '8804190'},
                           'res': {'type': 'literal',
                                   'value': 'New York is a state in the '
                                            'Northeastern United States. It '
                                            'was one of the original thirteen '
                                            'colonies forming the United '
                                            'States. With a total area of '
                                            '54,556 square miles (141,300 '
                                            'km2), New York is the 27th '
                                            'largest state geographically. Its '
                                            'population of more than 20 '
    

In [19]:
res_dict = result['results']['bindings']
for res_var in res_dict[0].keys():
    print(res_var," : ",res_dict[0][res_var]['value'])

res  :  New York is a state in the Northeastern United States. It was one of the original thirteen colonies forming the United States. With a total area of 54,556 square miles (141,300 km2), New York is the 27th largest state geographically. Its population of more than 20 million people makes it the fourth most populous state in the United States as of 2020. The state is bordered by New Jersey and Pennsylvania to the south, and Connecticut, Massachusetts, and Vermont to the east; it has a maritime border with Rhode Island, east of Long Island, as well as an international border with the Canadian provinces of Quebec to the north and Ontario to the northwest. It is sometimes called New York State to distinguish it from its largest city, New York City.
pop  :  8804190
